In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(4113, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(4113, 19)
Patient Number 	 289
State Patient Number 	 3296
Date Announced 	 394
Age Bracket 	 3307
Gender 	 3064
Detected City 	 3289
Detected District 	 1156
Detected State 	 394
State code 	 394
Current Status 	 394
Notes 	 1424
Contracted from which Patient (Suspected) 	 3189
Nationality 	 3686
Type of transmission 	 2285
Status Change Date 	 495
Source_1 	 587
Source_2 	 2753
Source_3 	 3843
Backup Notes 	 3753


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(4113, 19)


Patient Number                                289
State Patient Number                         3296
Date Announced                                394
Age Bracket                                  3307
Gender                                       3064
Detected City                                3289
Detected District                            1156
Detected State                                394
State code                                    394
Current Status                                394
Notes                                        1424
Contracted from which Patient (Suspected)    3189
Nationality                                  3686
Type of transmission                         2285
Status Change Date                            495
Source_1                                      587
Source_2                                     2753
Source_3                                     3843
Backup Notes                                 3753
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(3719, 19)


Patient Number                                  0
State Patient Number                         2902
Date Announced                                  0
Age Bracket                                  2913
Gender                                       2670
Detected City                                2895
Detected District                             762
Detected State                                  0
State code                                      0
Current Status                                  0
Notes                                        1030
Contracted from which Patient (Suspected)    2795
Nationality                                  3292
Type of transmission                         1891
Status Change Date                            101
Source_1                                      193
Source_2                                     2359
Source_3                                     3449
Backup Notes                                 3359
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
2505,2506,NaN,02/04/2020,NaN,NaN,NaN,NaN,Telangana,TG,Hospitalized,Details awaited,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124574147796092...,NaN,NaN,NaN
717,718,TN-P29,26/03/2020,65,F,NaN,Chennai,Tamil Nadu,TN,Hospitalized,Grandmother of TN-P28,P717,NaN,Local,26/03/2020,https://stopcoronatn.in/files/Media_Bulletin_2...,NaN,NaN,NaN
1477,1478,NaN,31/03/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,Details Awaited,NaN,NaN,TBD,31/03/2020,https://arogya.maharashtra.gov.in/pdf/epressno...,NaN,NaN,NaN
2662,2663,NaN,03/04/2020,NaN,NaN,NaN,NaN,Delhi,DL,Hospitalized,NaN,NaN,NaN,NaN,03/04/2020,https://twitter.com/ANI/status/124603857869975...,NaN,NaN,NaN
1456,1457,KA-P101,31/03/2020,62,F,Bengaluru,Bengaluru,Karnataka,KA,Hospitalized,Details Awaited,NaN,NaN,TBD,31/03/2020,https://twitter.com/DHFWKA/status/124500535143...,NaN,NaN,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
1378,1379,NaN,31/03/2020,NaN,NaN,NaN,Indore,Madhya Pradesh,MP,Hospitalized,Details awaited,NaN,NaN,TBD,31/03/2020,https://twitter.com/PTI_News/status/1244865476...,NaN,NaN,NaN
2558,2559,NaN,03/04/2020,NaN,NaN,NaN,Tonk,Rajasthan,RJ,Hospitalized,"Contact of Delhi Rel. Conf. Attendee, ID unknown",NaN,NaN,Local,03/04/2020,https://twitter.com/ANI/status/124593342855125...,NaN,NaN,NaN
3143,3144,NaN,04/04/2020,NaN,NaN,NaN,Udhampur,Jammu and Kashmir,JK,Hospitalized,Foreign Travel History,NaN,NaN,NaN,04/04/2020,https://twitter.com/kansalrohit69/status/12463...,NaN,NaN,NaN
1837,1838,TN-P142,01/04/2020,NaN,M,NaN,Madurai,Tamil Nadu,TN,Hospitalized,Attended Delhi Religious Conference,E0,NaN,NaN,01/04/2020,https://twitter.com/NHM_TN/status/124533236456...,NaN,NaN,NaN
197,198,DL-P13,19/03/2020,22,M,Ashok Vihar,North West Delhi,Delhi,DL,Hospitalized,Travelled from UK,NaN,India,Imported,19/03/2020,https://indianexpress.com/article/cities/delhi...,https://in.news.yahoo.com/four-coronavirus-cas...,NaN,Travel history to UK


In [19]:
p_df.to_csv('patients_data.csv', index=False)